In [125]:
import os

import numpy as np
import pandas as pd

from scipy import stats

import rioxarray

import xarray as xr
import geopandas as gpd

In [284]:
wb_path = "../../Bases_de_datos/CCKP_NetCDF/"
map_path = "../../Bases_de_datos/Mapas/WB_countries_Admin0_10m"
#map_path = "../../Bases_de_datos/Mapas/Natural_Earth/ne_10m_admin_0_countries"
wb_files = os.listdir(wb_path)
for i, f in enumerate(wb_files):
    if f[0] in ["~", "."]: wb_files.remove(f)
for i, f in enumerate(wb_files): print(f"{i}: {f}")

gdf = gpd.read_file(map_path)
gdf["Area"] = gdf.to_crs(8857).area
gdf#.plot()

0: climatology-popcount-annual-mean_pop-x0.25_gpw-v4-rev11-ssp245_climatology_mean_2040-2059.nc
1: climatology-spei12-annual-mean_cmip6-x0.25_ensemble-all-historical_climatology_median_1995-2014.nc
2: climatology-spei12-annual-mean_cmip6-x0.25_ensemble-all-ssp245_climatology_median_2040-2059.nc


,OBJECTID,featurecla,LEVEL,TYPE,FORMAL_EN,FORMAL_FR,POP_EST,POP_RANK,GDP_MD_EST,POP_YEAR,...,NAME_TR,NAME_VI,NAME_ZH,WB_NAME,WB_RULES,WB_REGION,Shape_Leng,Shape_Area,geometry,Area
0,1,Admin-0 country,2,Sovereign country,Republic of Indonesia,None,260580739,17,3028000.0,2017,...,Endonezya,Indonesia,印度尼西亚,Indonesia,None,EAP,495.029918,153.078608,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",1.879821e+12
1,2,Admin-0 country,2,Sovereign country,Malaysia,None,31381992,15,863000.0,2017,...,Malezya,Malaysia,马来西亚,Malaysia,None,EAP,68.456913,26.703172,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",3.278847e+11
2,3,Admin-0 country,2,Sovereign country,Republic of Chile,None,17789267,14,436100.0,2017,...,Şili,Chile,智利,Chile,None,LCR,416.997272,76.761813,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",7.365975e+11
3,4,Admin-0 country,2,Sovereign country,Plurinational State of Bolivia,None,11138234,14,78350.0,2017,...,Bolivya,Bolivia,玻利維亞,Bolivia,None,LCR,54.345991,92.203587,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",1.086808e+12
4,5,Admin-0 country,2,Sovereign country,Republic of Peru,None,31036656,15,410400.0,2017,...,Peru,Peru,秘鲁,Peru,None,LCR,73.262192,106.417089,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",1.289867e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,247,Admin-0 country,2,Dependency,None,None,300,2,0.0,0,...,United States Minor Outlying Islands,Các tiểu đảo xa của Hoa Kỳ,美国本土外小岛屿,Navassa Island (US),Name in italic,Other,0.085608,0.000413,"POLYGON ((-75.02432 18.41726, -75.02107 18.415...",4.829561e+06
245,248,Admin-0 country,2,Dependency,None,None,300,2,0.0,0,...,United States Minor Outlying Islands,Các tiểu đảo xa của Hoa Kỳ,美国本土外小岛屿,Palmyra Atoll (US),Name in italic,Other,0.147363,0.000576,"POLYGON ((-162.06086 5.88719, -162.05761 5.882...",7.051558e+06
246,249,Admin-0 country,2,Dependency,None,None,300,2,0.0,0,...,United States Minor Outlying Islands,Các tiểu đảo xa của Hoa Kỳ,美国本土外小岛屿,Kingman Reef (US),Name in italic,Other,0.059570,0.000222,"POLYGON ((-162.40018 6.44514, -162.38528 6.445...",2.713258e+06
247,250,Admin-0 country,2,Country,New Zealand,None,4510327,12,174800.0,2017,...,Yeni Zelanda,New Zealand,新西兰,Tokelau (NZ),Name in italic,Other,0.178453,0.000348,"MULTIPOLYGON (((-171.18566 -9.36126, -171.1883...",4.234951e+06


In [313]:
v = "climatology-popcount-annual-mean"
pop_iso = pop_f.copy()

gdf_b = gdf[gdf["Area"] >= 1e8]

for i in gdf_b.index:
    ds_c = xr.full_like( pop_f[["climatology-popcount-annual-mean"]],
        int(gdf_b.loc[gdf_b.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_b[gdf_b.index == i].geometry,
        drop = False, invert = False,all_touched = False )
    pop_iso[v] = pop_iso[v].where( pop_w[v].isnull(), pop_w[v] )

pop_iso

<xarray.Dataset> Size: 4MB
Dimensions:                           (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                              (time) datetime64[ns] 8B 2040-07-01
  * lat                               (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                               (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                              (bnds) int32 8B 0 1
    spatial_ref                       int64 8B 0
Data variables:
    climatology-popcount-annual-mean  (time, lat, lon) float32 4MB nan ... nan
    lat_bnds                          (lat, bnds) float64 12kB ...
    lon_bnds                          (lon, bnds) float64 23kB ...
Attributes: (12/20)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       2040-2059
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        gpw-v4
    ...                    ...
    Conventions:           CF-1.0
    Source_Software:       Esri ArcGIS
    creation_date:         Tue Nov 21 05:53:44 PM MST 2023
    user:                  file standardized by Caspar Ammann, Climate Strate...
    data_source:           Gridded Population of the World, Version 4: GPWv4;...
    title:                 Gridded Population of the World, Version 4, Revisi...

In [314]:
iso = pop_iso[v].values
iso_c  = iso.copy()
for i in range(iso.shape[1]):
    print(f"{i+1}/{iso.shape[1]}")
    for j in range(iso.shape[2]):

        i_l = i - 1
        i_p = i + 1
        j_l = j - 1
        j_p = j + 1

        if i_l < 0: i_l = iso.shape[1] - 1
        if i_p == iso.shape[1]: i_p = 0
        if j_l < 0: j_l = iso.shape[2] - 1
        if j_p == iso.shape[2]: j_p = 0

        adj = [0] * 8
        adj[0] = iso[0, i_l, j_p]
        adj[1] = iso[0, i, j_p]
        adj[2] = iso[0, i_p, j_p]
        adj[3] = iso[0, i_l, j]
        cell =  iso[0, i, j]
        adj[4] = iso[0, i_p, j]
        adj[5] = iso[0, i_l, j_l]
        adj[6] = iso[0, i, j_l]
        adj[7] = iso[0, i_p, j_l]

        if np.isnan(cell) or (cell in gdf["ISO_N3"].astype(int).values): pass
        else:
            adj_c = []
            for k in adj:
                if k in gdf["ISO_N3"]: adj_c.append(k)
                if len(adj_c) > 0:
                    m = stats.mode(adj_c)[0]
                    if not (type(m) == np.float32): m = m[0]
                    iso_c[0, i, j] = int(m)
        
        cell = iso_c[0, i, j]
        if np.isnan(cell) or (cell in gdf["ISO_N3"].astype(int).values): pass
        #else: iso_c[0, i, j] = np.nan

        #if iso[0, i, j] == 360:
        #    print(iso[0, i, j] in gdf["ISO_N3"].astype(int))
        #    print(iso_c[0, i, j])

pop_iso[v] = ( ("time", "lat", "lon"), iso_c )

1/721
2/721
3/721
4/721
5/721
6/721
7/721
8/721
9/721
10/721
11/721
12/721
13/721
14/721
15/721
16/721
17/721
18/721
19/721
20/721
21/721
22/721
23/721
24/721
25/721
26/721
27/721
28/721
29/721
30/721
31/721
32/721
33/721
34/721
35/721
36/721
37/721
38/721
39/721
40/721
41/721
42/721
43/721
44/721
45/721
46/721
47/721
48/721
49/721
50/721
51/721
52/721
53/721
54/721
55/721
56/721
57/721
58/721
59/721
60/721
61/721
62/721
63/721
64/721
65/721
66/721
67/721
68/721
69/721
70/721
71/721
72/721
73/721
74/721
75/721
76/721
77/721
78/721
79/721
80/721
81/721
82/721
83/721
84/721
85/721
86/721
87/721
88/721
89/721
90/721
91/721
92/721
93/721
94/721
95/721
96/721
97/721
98/721
99/721
100/721
101/721
102/721
103/721
104/721
105/721
106/721
107/721
108/721
109/721
110/721
111/721
112/721
113/721
114/721
115/721
116/721
117/721
118/721
119/721
120/721
121/721
122/721
123/721
124/721
125/721
126/721
127/721
128/721
129/721
130/721
131/721
132/721
133/721
134/721
135/721
136/721
137/721
138/721
139/

In [315]:
v = "climatology-popcount-annual-mean"

gdf_s = gdf[gdf["Area"] < 1e8]

for i in gdf_s.index: 
    ds_c = xr.full_like( pop_f[["climatology-popcount-annual-mean"]],
        int(gdf_s.loc[gdf_s.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_s[gdf_s.index == i].geometry,
        drop = False, invert = False, all_touched = True )
    pop_iso[v] = pop_iso[v].where( pop_w[v].isnull(), pop_w[v] )

iso = pop_iso[v].values
iso_c  = iso.copy()

for i in range(iso.shape[1]):
    for j in range(iso.shape[2]):
        cell = iso[0, i, j]
        if np.isnan(cell) or (cell in gdf["ISO_N3"].astype(int).values): pass
        else: iso_c[0, i, j] = np.nan

pop_iso[v] = ( ("time", "lat", "lon"), iso_c )

pop_iso

<xarray.Dataset> Size: 4MB
Dimensions:                           (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                              (time) datetime64[ns] 8B 2040-07-01
  * lat                               (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                               (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                              (bnds) int32 8B 0 1
    spatial_ref                       int64 8B 0
Data variables:
    climatology-popcount-annual-mean  (time, lat, lon) float32 4MB nan ... nan
    lat_bnds                          (lat, bnds) float64 12kB ...
    lon_bnds                          (lon, bnds) float64 23kB ...
Attributes: (12/20)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       2040-2059
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        gpw-v4
    ...                    ...
    Conventions:           CF-1.0
    Source_Software:       Esri ArcGIS
    creation_date:         Tue Nov 21 05:53:44 PM MST 2023
    user:                  file standardized by Caspar Ammann, Climate Strate...
    data_source:           Gridded Population of the World, Version 4: GPWv4;...
    title:                 Gridded Population of the World, Version 4, Revisi...

In [316]:
iso_c = pop_iso[v].values

In [317]:
np.unique(iso_c)

array([-99.,   4.,   8.,  12.,  16.,  20.,  24.,  28.,  31.,  32.,  36.,
        40.,  44.,  48.,  50.,  51.,  56.,  60.,  64.,  68.,  70.,  72.,
        76.,  84.,  86.,  90.,  96., 100., 104., 108., 112., 116., 120.,
       124., 132., 136., 140., 144., 148., 152., 156., 162., 166., 170.,
       174., 178., 180., 184., 188., 191., 192., 196., 203., 204., 208.,
       214., 218., 222., 226., 231., 232., 233., 234., 238., 239., 242.,
       246., 250., 258., 260., 262., 266., 268., 270., 275., 276., 288.,
       292., 296., 300., 304., 320., 324., 328., 332., 334., 336., 340.,
       344., 348., 352., 356., 360., 364., 368., 372., 376., 380., 384.,
       388., 392., 398., 400., 404., 408., 410., 414., 417., 418., 422.,
       426., 428., 430., 434., 440., 442., 446., 450., 454., 458., 466.,
       478., 480., 484., 492., 496., 498., 499., 500., 504., 508., 512.,
       516., 520., 524., 528., 533., 534., 540., 548., 554., 558., 562.,
       566., 574., 578., 581., 586., 591., 598., 60

In [318]:
np.unique(iso_c)[1:-1].shape

(215,)

In [291]:
gdf[ ~gdf["ISO_N3"].astype(int).isin(np.unique(iso_c)[1:-1]) ].to_crs(8857)[["ISO_N3", "FORMAL_EN"]]

,ISO_N3,FORMAL_EN
62,-99,Republic of Kosovo
91,438,Principality of Liechtenstein
169,531,Curaçao
172,796,Turks and Caicos Islands
174,666,Saint Pierre and Miquelon
181,584,Republic of the Marshall Islands
183,308,Grenada
185,052,Barbados
187,212,Commonwealth of Dominica
190,659,Federation of Saint Kitts and Nevis


In [295]:
iso[0, 481, 412]

nan

In [206]:
gdf[ gdf["ISO_N3"].astype(int).isin(np.unique(iso_c)[1:-1]) ].to_crs(8857).area.sort_values()#[["ISO_N3", "FORMAL_EN"]]

249    4.234951e+06
236    7.064307e+06
235    1.903438e+07
195    8.067597e+07
171    1.697672e+08
           ...     
42     8.472665e+12
8      9.380775e+12
149    9.464325e+12
150    9.945523e+12
45     1.087259e+13
Length: 199, dtype: float64

In [205]:
gdf.loc[50]

OBJECTID                                                     51
featurecla                                      Admin-0 country
LEVEL                                                         2
TYPE                                          Sovereign country
FORMAL_EN                                     Kingdom of Norway
FORMAL_FR                                                  None
POP_EST                                                 5320045
POP_RANK                                                     13
GDP_MD_EST                                             364700.0
POP_YEAR                                                   2017
LASTCENSUS                                                 2001
GDP_YEAR                                                   2016
ECONOMY                              2. Developed region: nonG7
INCOME_GRP                                 1. High income: OECD
FIPS_10_                                                    -99
ISO_A2                                  

In [208]:
gdf.to_crs(8857).area.sort_values().tail(50)

103    5.790294e+11
15     5.823185e+11
167    5.929802e+11
31     5.990942e+11
82     6.179835e+11
13     6.302407e+11
20     6.367572e+11
14     6.392213e+11
125    6.421811e+11
122    6.631595e+11
2      7.365975e+11
78     7.519147e+11
63     7.800801e+11
118    7.884476e+11
33     8.227136e+11
16     8.729388e+11
96     9.074985e+11
154    9.126840e+11
18     9.415058e+11
156    1.001081e+12
158    1.036389e+12
165    1.064293e+12
3      1.086808e+12
12     1.127376e+12
133    1.135118e+12
143    1.181302e+12
34     1.219825e+12
98     1.244652e+12
94     1.252723e+12
110    1.266282e+12
4      1.289867e+12
44     1.564660e+12
90     1.622510e+12
69     1.623757e+12
83     1.846762e+12
0      1.879821e+12
102    1.921724e+12
151    1.957848e+12
164    2.154380e+12
117    2.308858e+12
29     2.325240e+12
39     2.720765e+12
5      2.784303e+12
7      3.082421e+12
163    7.691177e+12
42     8.472665e+12
8      9.380775e+12
149    9.464325e+12
150    9.945523e+12
45     1.087259e+13


In [94]:
i = 0
pop_f = xr.open_dataset(wb_path + wb_files[0])
pop_f = pop_f.rio.write_crs(gdf.crs)

i = 2
ds_f = xr.open_dataset(wb_path + wb_files[2])
ds_f = ds_f.where( ds_f["climatology-spei12-annual-mean"] <= -0.25, 0 )
ds_f = ds_f.where( ds_f["climatology-spei12-annual-mean"] == 0, 1 )

ds_f["Exposed population"] = ( pop_f["climatology-popcount-annual-mean"]
    * ds_f["climatology-spei12-annual-mean"] )
ds_f["Exposed population"]

<xarray.DataArray 'Exposed population' (time: 1, lat: 721, lon: 1440)> Size: 4MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 8B 2040-07-01
  * lat          (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon          (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
    spatial_ref  int64 8B 0